In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [6]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [7]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [8]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=1)
update_tree_flg = False

## train & validate model

In [9]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, rads_bow_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
            if update_tree_flg:
                print(config.tree_idxs)
                name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                if 'sess' in globals(): sess.close()
                model = HierarchicalNeuralTopicModel(config)
                sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, rads_bow_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                           VALID:               \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS  PPL     NLL   
5000     66   10    9  111.93  509  111.17  0.75  0.01  104.15  470  103.09   
10000    68   20   19  111.38  486  110.31  1.04  0.03  103.92  454  102.53   
15000    67   30   29  111.11  473  109.85  1.22  0.04  103.77  446  102.25   
20000    50   40   39  110.94  466  109.58  1.33  0.03  103.72  445  102.20   
25000    54   50   49  110.81  460  109.36  1.42  0.03  103.61  437  101.94   
30000    60   60   59  110.71  455  109.18  1.50  0.03  103.49  431  101.74   
35000    63   70   69  110.63  452  109.03  1.56  0.03  103.46  426  101.58   
40000    50   80   79  110.57  449  108.93  1.61  0.03  103.45  427  101.64   
45000    60   90   89  110.51  446  108.82  1.66  0.03  103.35  422  101.46   
50000    62  100   99  110.46  444  108.73  1.70  0.03  103.32  419  101.33   
55000    54  110  109  110.42  442  108.66  1.73  0.03  103.31  420  101.40   
60000    65  120  119  110.38  440  108.58  1.77  0.03  103.23  415  101.22   
65000    66  130  129  110.34  438  108.51  1.80  0.03  103.22  413  101.12   
70000    54  140  139  110.31  437  108.46  1.82  0.03  103.26  417  101.24   
75000    64  150  149  110.28  436  108.41  1.85  0.03  103.18  413  101.10   
80000    67  160  159  110.26  434  108.36  1.87  0.03  103.19  411  101.04   
85000    54  170  169  110.24  433  108.32  1.89  0.03  103.21  414  101.16   
90000    77  180  179  110.21  432  108.28  1.91  0.02  103.14  410  101.01   
95000    69  190  189  110.19  431  108.24  1.93  0.03  103.33  414  101.14   
100000   55  200  199  110.18  431  108.21  1.94  0.02  103.29  416  101.23   
105000   64  210  209  110.16  430  108.18  1.96  0.02  103.18  411  101.04   
110000   67  220  219  110.15  429  108.14  1.98  0.02  103.20  410  101.00   
115000   54  230  229  110.14  428  108.12  1.99  0.02  103.25  414  101.15   
120000   65  240  239  110.12  428  108.09  2.00  0.02  103.17  410  101.01   
125000   66  250  249  110.11  427  108.07  2.01  0.02  103.16  407  100.91   
130000   54  260  259  110.10  426  108.05  2.02  0.02  103.19  411  101.08   
135000   65  270  269  110.08  426  108.02  2.04  0.02  103.11  407  100.92   
140000   67  280  279  110.07  425  108.00  2.05  0.02  103.11  406  100.85   
145000   57  290  289  110.06  425  107.98  2.06  0.02  103.18  410  101.03   
150000   65  300  299  110.05  424  107.96  2.07  0.02  103.10  406  100.87   
...     ...  ...  ...     ...  ...     ...   ...   ...     ...  ...     ...   
350000   61  701  200  109.81  413  107.50  2.29  0.01  103.02  401  100.66   
355000   60  711  210  109.80  413  107.49  2.29  0.01  103.01  400  100.63   
360000   60  721  220  109.80  412  107.49  2.30  0.01  103.01  400  100.63   
365000   61  731  230  109.80  412  107.48  2.30  0.01  103.00  400  100.62   
370000   61  741  240  109.79  412  107.47  2.30  0.01  103.00  400  100.63   
375000   61  751  250  109.79  412  107.47  2.30  0.01  102.99  399  100.61   
380000   63  761  260  109.79  412  107.46  2.31  0.01  102.97  399  100.58   
385000   61  771  270  109.78  412  107.46  2.31  0.01  102.97  399  100.59   
390000   59  781  280  109.78  411  107.45  2.31  0.01  102.95  398  100.56   
395000   62  791  290  109.78  411  107.45  2.32  0.01  102.97  399  100.57   
400000   66  801  300  109.77  411  107.44  2.32  0.01  102.97  399  100.57   
405000   71  811  310  109.77  411  107.44  2.32  0.01  102.98  399  100.58   
410000   61  821  320  109.77  411  107.43  2.32  0.01  102.98  399  100.59   
415000   61  831  330  109.77  411  107.43  2.33  0.01  102.98  399  100.59   
420000   61  841  340  109.76  411  107.42  2.33  0.01  103.00  399  100.60   
425000   61  851  350  109.76  411  107.42  2.33  0.01  103.00  400  100.60   
430000   67  861  360  109.76  410  107.41  2.33  0.01  103.00  400  100.60   
435000   59  871  370  109.75  410  107.41  2.33  0.01  102.99  399  100.

0 R: 1.000 P: 0.401 quality price bought 'm time ... buy 've nice back
   1 R: 0.298 P: 0.116 strap pocket pockets side shoulder inside compartment zipper small nice
     11 R: 0.089 P: 0.089 carry pockets room compartments pack comfortable plenty back books space
     12 R: 0.040 P: 0.040 mouse power netbook cord pocket drive room usb cords accessories
     14 R: 0.053 P: 0.053 ipad carry room charger perfect pocket small love extra front
   6 R: 0.132 P: 0.074 sleeve protection air pro inside smell protect snug inch zipper
     62 R: 0.022 P: 0.022 & ; perfectly size hp big tablet laptops dell room
     63 R: 0.035 P: 0.035 sleeve perfectly chromebook protection inch inside protect tablet soft perfect
   4 R: 0.064 P: 0.023 mac ! pro love air book perfectly recommend protect easy
     42 R: 0.041 P: 0.041 ! love perfect recommend color absolutely loves buy ... awesome
   2 R: 0.105 P: 0.058 bottom cover top scratches plastic speck easily apple hard feet
     21 R: 0.047 P: 0.047 colo

TRAIN:                           VALID:               \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS  PPL     NLL   
5000     66   10    9  111.93  509  111.17  0.75  0.01  104.15  470  103.09   
10000    68   20   19  111.38  486  110.31  1.04  0.03  103.92  454  102.53   
15000    67   30   29  111.11  473  109.85  1.22  0.04  103.77  446  102.25   
20000    50   40   39  110.94  466  109.58  1.33  0.03  103.72  445  102.20   
25000    54   50   49  110.81  460  109.36  1.42  0.03  103.61  437  101.94   
30000    60   60   59  110.71  455  109.18  1.50  0.03  103.49  431  101.74   
35000    63   70   69  110.63  452  109.03  1.56  0.03  103.46  426  101.58   
40000    50   80   79  110.57  449  108.93  1.61  0.03  103.45  427  101.64   
45000    60   90   89  110.51  446  108.82  1.66  0.03  103.35  422  101.46   
50000    62  100   99  110.46  444  108.73  1.70  0.03  103.32  419  101.33   
55000    54  110  109  110.42  442  108.66  1.73  0.03  103.31  420  101.40   
60000    65  120  119  110.38  440  108.58  1.77  0.03  103.23  415  101.22   
65000    66  130  129  110.34  438  108.51  1.80  0.03  103.22  413  101.12   
70000    54  140  139  110.31  437  108.46  1.82  0.03  103.26  417  101.24   
75000    64  150  149  110.28  436  108.41  1.85  0.03  103.18  413  101.10   
80000    67  160  159  110.26  434  108.36  1.87  0.03  103.19  411  101.04   
85000    54  170  169  110.24  433  108.32  1.89  0.03  103.21  414  101.16   
90000    77  180  179  110.21  432  108.28  1.91  0.02  103.14  410  101.01   
95000    69  190  189  110.19  431  108.24  1.93  0.03  103.33  414  101.14   
100000   55  200  199  110.18  431  108.21  1.94  0.02  103.29  416  101.23   
105000   64  210  209  110.16  430  108.18  1.96  0.02  103.18  411  101.04   
110000   67  220  219  110.15  429  108.14  1.98  0.02  103.20  410  101.00   
115000   54  230  229  110.14  428  108.12  1.99  0.02  103.25  414  101.15   
120000   65  240  239  110.12  428  108.09  2.00  0.02  103.17  410  101.01   
125000   66  250  249  110.11  427  108.07  2.01  0.02  103.16  407  100.91   
130000   54  260  259  110.10  426  108.05  2.02  0.02  103.19  411  101.08   
135000   65  270  269  110.08  426  108.02  2.04  0.02  103.11  407  100.92   
140000   67  280  279  110.07  425  108.00  2.05  0.02  103.11  406  100.85   
145000   57  290  289  110.06  425  107.98  2.06  0.02  103.18  410  101.03   
150000   65  300  299  110.05  424  107.96  2.07  0.02  103.10  406  100.87   
...     ...  ...  ...     ...  ...     ...   ...   ...     ...  ...     ...   
350000   61  701  200  109.81  413  107.50  2.29  0.01  103.02  401  100.66   
355000   60  711  210  109.80  413  107.49  2.29  0.01  103.01  400  100.63   
360000   60  721  220  109.80  412  107.49  2.30  0.01  103.01  400  100.63   
365000   61  731  230  109.80  412  107.48  2.30  0.01  103.00  400  100.62   
370000   61  741  240  109.79  412  107.47  2.30  0.01  103.00  400  100.63   
375000   61  751  250  109.79  412  107.47  2.30  0.01  102.99  399  100.61   
380000   63  761  260  109.79  412  107.46  2.31  0.01  102.97  399  100.58   
385000   61  771  270  109.78  412  107.46  2.31  0.01  102.97  399  100.59   
390000   59  781  280  109.78  411  107.45  2.31  0.01  102.95  398  100.56   
395000   62  791  290  109.78  411  107.45  2.32  0.01  102.97  399  100.57   
400000   66  801  300  109.77  411  107.44  2.32  0.01  102.97  399  100.57   
405000   71  811  310  109.77  411  107.44  2.32  0.01  102.98  399  100.58   
410000   61  821  320  109.77  411  107.43  2.32  0.01  102.98  399  100.59   
415000   61  831  330  109.77  411  107.43  2.33  0.01  102.98  399  100.59   
420000   61  841  340  109.76  411  107.42  2.33  0.01  103.00  399  100.60   
425000   61  851  350  109.76  411  107.42  2.33  0.01  103.00  400  100.60   
430000   67  861  360  109.76  410  107.41  2.33  0.01  103.00  400  100.60   
435000   59  871  370  109.75  410  107.41  2.33  0.01  102.99  399  100.

0 R: 1.000 P: 0.408 quality price bought 'm time ... buy 've nice back
   1 R: 0.287 P: 0.112 strap pocket pockets side shoulder inside small zipper compartment nice
     11 R: 0.086 P: 0.086 carry pockets room compartments pack comfortable back space plenty books
     14 R: 0.043 P: 0.043 mouse power netbook cord pocket drive room usb cords accessories
     13 R: 0.045 P: 0.045 ipad room carry charger perfect pocket small love extra perfectly
   6 R: 0.131 P: 0.071 sleeve protection air pro inside smell protect snug inch zipper
     63 R: 0.021 P: 0.021 ; & perfectly size hp big tablet laptops dell room
     61 R: 0.038 P: 0.038 sleeve chromebook perfectly protection inch inside protect soft perfect tablet
   2 R: 0.063 P: 0.021 mac ! love pro air book perfectly recommend protect easy
     22 R: 0.042 P: 0.042 ! love perfect recommend color absolutely loves buy ... awesome
   3 R: 0.111 P: 0.058 bottom cover top scratches plastic easily speck apple hard feet
     31 R: 0.052 P: 0.052 